# **Micro-Projet Biostatistiques**

Antoine Bridier-Nahmias et Claire Vandiedonck - UE3 "Analyses de données" de la mineure "Recherche en Santé" - Université Paris Cité

<mark>Notebook à positionner dans le repretoire <code>~/pass_minrs_ue3/micro_projet_bi/</code></mark>

## Polymorphisme dans _BRCA1_ à partir des données du projet 1000 genomes

Ce micro-projet a pour objectif d'explorer la variabilité génétique du gène _BRCA1_ au moyen des données du projet  ['1000 genomes'](https://www.internationalgenome.org/home).<br>
Il fait suite au micro-projet Unix dans lequel nous avions récupéré le fichier `.vcf` (pour *variant calling format)* de variants du chromosome 17 du projet 1000 genomes. <br>
Dans ce micro-projet, nous utiliserons un sous-fichier `.vcf` d'une portion du chromosome 17. Ce fichier a été **annoté** au moyen du logiciel [ANNOVAR](https://annovar.openbioinformatics.org/en/latest/). Les annotations ainsi ajoutées dans les 1ères collones (avant les génotypes) incluent:
- l'identifiant `rsID` de chaque variant dans la base de données [dbSNP](https://www.ncbi.nlm.nih.gov/snp/) du ncbi
- la position de chaque variant relative aux différents gènes
- l'impact fonctionnel putatif de chaque variant sur la protéine
- des données de fréquences

Notre objectif sera, avec le langage R:<br>
1. d'étudier la distribution des variants en fonction de leur position le long du gène *BRCA1*
2. d'explorer le degré d'hétérozygotie de ces variants
3. d'évaluer la pathogénicité de ces variants
4. d'explorer la variabilité génétique de *BRCA1* selon les populations

<div class="alert alert-warning"><b> Modalités d'évaluation :</b><br><br>
Les boîtes oranges contiennent des instructions que vous allez devoir traduire en commandes R.
Une partie de la commande peut vous être donnée, à vous de la compléter en remplacant les <b>[XXX]</b>.
<br/>
<br/>
    L'évaluation consistera en un simple questionnaire numéroté sur <b>moodle</b> dans lequel vous devrez reporter ce par quoi vous avez remplacé ces <b>[XXX]</b>, le résultat de certaines commandes ou encore des champs de réponses libres.
<br/>
<br/>
Contrairement à ce qui vous avait initialement été indiqué, le travail sera évalué <u><b>individuellement et NON en binôme</b></u>, le format du micro-projet ayant été simplifié. La date limite pour rendre vos réponses sur moodle est fixée au <u><b>10/05/2023 20h00</b></u>.
</div>

Avant de commencer, on se positionne dans le bon repertoire de travail, on affiche la version de R utilisée et les paquets chargés dans la session. 

In [ ]:
##### cell 1
setwd("~/pass_minrs_ue3/micro_projet_bi/")
getwd()
library(tidyverse)
library(FactoMineR)
library(factoextra)
library(ggpubr)
sessionInfo()

## I. Chargement des données
---

### I.A. Chargement du .vcf

On charge d'abord le fichier tabulé `.vcf` annoté avec ANNOVAR. Il a été mis dans le répertoire `/srv/data/pass-rs-ue3/brca1/` sur adenine. <mark>Cette commande peut prendre quelques secondes.</mark>

In [ ]:
##### cell 2
vcf_annot <-  read.table('/srv/data/pass-rs-ue3/brca1/chr17_brca1.tsv',
                         header = TRUE,
                         sep = "\t")
str(vcf_annot)

### I.B. Chargement des metadata

On charge à présent un fichier de données supplémentaires (*metadata*) donnant des indications sur les sujets inclus dans le projet 1000 genomes, en particulier leur sexe et leur origine géographique. Ce fichier est dans le répertoire relatif au chromosome 17 sur adenine.

In [ ]:
##### cell 3
metadata <- read.table('/srv/data/pass-rs-ue3/hg38-chr17-1kgenome/20130606_g1k_3202_samples_ped_population.txt',
                       sep=" ",
                       header = TRUE)
str(metadata)

=> Nous avons à présent deux objets chargés dans notre session R, comme l'indique le résultat de la commande `ls()`

In [ ]:
##### cell 4
ls()

## II. Un premier regard sur les données de variants...
---


### II.A. Dimensions du fichier .vcf annoté

Chaque ligne représente un SNV (*Single Nucleotide Variant*) et les sujets font partie des variables mises en colonnes.

In [ ]:
##### cell 5
dim(vcf_annot)
head(vcf_annot)

Compte tenu des dimensions de `vcf_annot`, dans un notebook jupyter, l'affichage de toutes les colonnes est en partie masqué lorsque vous utilisez la fonction `head()` ou `str()`. Vous pouvez aussi afficher les noms des colonnes avec la fonction `names()`. Là encore l'affichage sera réduit dans un notebook Jupyter mais vous pouvez lire plus de colonnes.

In [ ]:
##### cell 6
names(vcf_annot)

<div class="alert alert-warning"> 
    <p>
        <b>Question 1)</b> A propos de <code>vcf_annot</code> :<br>
           &emsp; Le dataset contient les informations de 2697 SNVs ? 
            <br>
           &emsp; Le dataset contient les informations de 4918 sujets ?
            <br>
           &emsp; Le dataset contient les informations de 4918 SNVs ? 
            <br>
           &emsp; Le dataset contient les informations de 2697 sujets ?
            <br>
           &emsp; Le dataset contient les informations de moins de 2697 sujets ?
    </p>
</div>

### II.B. Gènes présents dans le fichier .vcf annoté

Les colonnes d'annotations relatives aux gènes sont les suivantes:

In [ ]:
##### cell 7
gene_columns <- grep("Gene", names(vcf_annot))
names(vcf_annot)[gene_columns]

=> Il existe plusieurs versions d'annotations des gènes sur le génome (positions des gènes).  Dans le fichier `.vcf` annoté, les colonnes dont le nom se termine par `.refGene` sont celles de l'annotation [RefSeq](https://www.ncbi.nlm.nih.gov/refseq/) du génome, tandis que les colonnes se terminant par `.ensGene` sont celles de l'annotation [ENSEMBL](https://www.ensembl.org/index.html) du génome. Le symbole du gène est disponible dans les colonnes `Gene.refGene` ou `Gene.ensGene`. Pour la suite de ce micro-projet, nous utiliserons exclusivement les annotations de ***RefSeq***.

Nous pouvons lister toutes les co-occurences uniques de noms de gènes dans notre fichier de variants avec la fonction `unique()` ou la fonction `table()` qui permet en plus de compter le nombre d'occurences par gène.

In [ ]:
##### cell 8
unique(vcf_annot$Gene.refGene)
table(vcf_annot$Gene.refGene, useNA = "always")

### II.C. Création d'un sous-jeu de données pour le gène *BRCA1*

Par souci de simplicité pour ce microprojet, nous nous focalisons sur les variants pour lesquels le gène annoté est exclusivement *BRCA1*.

Nous réalisons donc un sous-jeu de données pour ce gène avec la fonction `subset()`.

In [ ]:
##### cell 9

# Subsetting to keep BRCA1 only:
brca1_vcf <- subset(vcf_annot, Gene.refGene == 'BRCA1')
dim(brca1_vcf)

<div class="alert alert-warning"> 
    <b>Question 2) :</b>
    Le dataset ne contient-il désormais que des SNP présents dans <i>BRCA1</i> (vous pouvez vous aider de la commande <code>table()</code> ou d'une autre commande de votre choix).
</div>

<div class="alert alert-warning"> 
    <b>Question 3) :</b>
    Quelle commande avez-vous utilisé pour vous assurer que le dataframe `brca1_vcf` ne contenait que des SNP présents dans <i>BRCA1</i> ? (plusieurs commandes et écritures sont possibles, entrez la commande exacte que vous avez éxécutée).
</div>

In [ ]:
##### cell 10

# Votre commande:


### II.D. Distribution des variants le long du gène *BRCA1*

Nous explorons à présent les données avec différentes représentations graphiques.

In [ ]:
##### cell 11

# SNP distribution on BRCA1:
snp_density <- density(brca1_vcf$Start,
                       adjust = 0.8)

# plot the snp density
plot(x = snp_density,
     frame = FALSE,
     main = "SNP density across BRCA1")

# secondary fonction to color the area under the curve
polygon(snp_density,
        col = "steelblue") 

In [ ]:
##### cell 12
snp_density <- density(brca1_vcf$Start,
                       adjust = 0.2)

plot(x = snp_density,
     frame = FALSE,
     main = "SNP density across BRCA1")

polygon(snp_density,
        col = "steelblue")

In [ ]:
##### cell 13
hist(x = brca1_vcf$Start,
     main = "SNP density in BRCA1",
     breaks = 1e2,
     col = "steelblue",
     freq = FALSE,
     xlab = "SNV coordinates on chr17")

<div class="alert alert-warning"> 
    <b>Question 4) :</b>
        <br>
        &emsp;Ces plots représentent la densité en SNP le long du gène <i>BRCA1</i>. 
        <br>
        &emsp;Laquelle de ces représentations choisiriez-vous et pourquoi (justifiez en une brève phrase) ?
</div>

## III. Homo et hétéro-zygotie
---

### III.A. Génération d'une variable discrète *genotype* { 0, 1, 2 } indiquant le nombre de copies de l'allèle alternatif à l'allèle de référence sur la séquence du génome pour chaque individu. 

Dans le fichier `.vcf` les 149 1ères colonnes fournissent des annotations sur les SNVs, les suivantes donnent les génotypes phasés de chaque sujet du projet 1000 genomes.
Nous allons extraire les colonnes correspondantes dans 2 nouveaux objets R.

- **un dataframe pour les informations sur les variants :**

In [ ]:
##### cell 14

# separating SNP and genotypes
brca1_snps <-  brca1_vcf[ , 1:149 ]

- **une matrice pour les informations sur les génotypes** avec une écriture simplifiée :

On opère en trois temps :

1. d'abord en extrayant les colonnes d'intérêt dans un sous-dataframe.

In [ ]:
##### cell 15

# separating SNP and genotypes
brca1_samples <- brca1_vcf[ , 150:ncol(brca1_vcf) ]

2. Puis en créant une fonction pour automatiquement remplacer les génotypes phasés par le nombre de copies de l'allèle alternatif.

In [ ]:
##### cell 16

# This function transforms the 0|0 , 1|0, 0|1 and 1|1 into a more numerical variable
# 0 is 0|0
# 1 is 0|1 or 1|0
# 2 is 1|1 

simplify_genotype <- function(genotype) {
    genotype <- unlist(genotype)
    out <- vector(mode = 'numeric', length = length(genotype))
    out[genotype == '0|0'] <- 0
    out[genotype == '1|0'] <- 1
    out[genotype == '0|1'] <- 1
    out[genotype == '1|1'] <- 2
    # out <- factor(x = out, levels = c(0, 1, 2), labels = c(0, 1, 2))
    return(out)
}

3. Et enfin en appliquant cette fonction sur le sous-dataframe et en obtenant une matrice de valeurs 0, 1 ou 2.

In [ ]:
##### cell 17

# apply the transformation
brca1_genotypes <-  sapply(brca1_samples,
                            simplify_genotype,
                            simplify = TRUE)
str(brca1_genotypes)
brca1_genotypes[1:20, 1:20] # on affiche ici les 20 premiers SNVs pour les 20 premiers sujets

### III.B. Comptes et plots

- Nous allons compter les **effectifs et proportion de chaque type de génotype sur l'ensemble des sujets et des variants** et faire une réprésentation graphique *<mark>(la cellule suivante prend quelques secondes à s'executer)</mark>*:

In [ ]:
##### cell 18
table(brca1_genotypes)
proportions(as.matrix(table(brca1_genotypes)), 2)
barplot(proportions (as.matrix(table(brca1_genotypes)), 2),
    beside = TRUE, legend = 0:2)

Nous voyons que la majorité des génotypes sont homozygotes pour l'allèle de référence. Mais qu'en est-il pour chaque sujet et chaque SNV? 

- Nous allons à présent compter les **effectifs de chaque type de génotype par sujet**:

La fonction `apply()` avec l'argument `MARGIN = 2` permet d'appliquer une même fonction sur chaque ligne de l'objet X, ici générant une variable pour chaque génotype, contenant le comptes de variants par sujet.

In [ ]:
##### cell 19

SNV_count_homozy_REF <- apply( X = brca1_genotypes,
                    MARGIN = 2,
                    FUN = function(x) sum(x == 0))
                              
SNV_count_heterozy <- apply(X = brca1_genotypes,
                  MARGIN = 2,
                  FUN = function(x) sum(x == 1))
                  
SNV_count_homozy_ALT <- apply(X = brca1_genotypes,
                    MARGIN = 2,
                    FUN = function(x) sum(x == 2))                    

Nous regroupons ces résultats dans un nouveau dataframe **SNV_count_zygo_df**, avec chaque type de génotype dans une colonne comme une variable quantitative séparée. Chaque ligne correspond ici à un sujet.

In [ ]:
##### cell 20
SNV_count_zygo_df <- data.frame(SNV_count_homozy_REF, SNV_count_heterozy, SNV_count_homozy_ALT)
str(SNV_count_zygo_df)
head(SNV_count_zygo_df)

Dans le dataframe ci-dessus, chaque ligne donne le nombre de SNPs par catégorie de génotype pour  l'individu de la ligne. A noter que pour chaque ligne, la somme des comptes (calculée avec la fonction `rowSums()`) donne le nombre de variants par individu, et que cette somme est la même pour chaque ligne comme l'indique le résumé ci-dessous.

In [ ]:
##### cell 21
summary(rowSums(SNV_count_zygo_df))

Dans le dataframe `SNV_count_zygo_df`, chaque colonne est une variable discrète. Ces 3 variables ne sont pas indépendantes car leur somme correspond au nombre de variants.

On peut regarder les valeurs de dispersion :

In [ ]:
##### cell 22
summary(SNV_count_zygo_df)

et représenter leur distribution avec un barplot, par exemple ci-dessous avec la variable sur le compte de SNVs homozygotes :

In [ ]:
##### cell 23
barplot(table(SNV_count_homozy_REF))

Compte-tenu du nombre élevé de valeurs possibles pour chacune de ces variables {0, 1, ..., 1969}, on peut assimiler ces variables discrètes à des variables quantitatives continues et explorer aussi leur distribution avec un histogramme (en jouant sur le nombre d'intervalles) ou avec un boxplot par exemple.

In [ ]:
##### cell 24
hist(SNV_count_homozy_REF,
     ylab = "nombre de sujets",
     freq = TRUE,
     breaks = 1000 )

hist(SNV_count_homozy_REF,
     ylab = "nombre de sujets",
     freq = TRUE,
     breaks = 100 ) 

On représente maintenant les 3 histogrammes côte à côte dans un même graphique.

In [ ]:
##### cell 25
options(repr.plot.width = 20, repr.plot.height = 10)
opar <- par()
par(mfrow = c(1, 3))
hist(SNV_count_homozy_REF,
     ylab = "nombre de sujets",
     freq = TRUE,
     breaks = 100,
     main = "Distribution des comptes de SNVs \n homoyzgotes pour l'allèle de référence",
     cex.lab = 1.5,
     cex.main = 2)
hist(SNV_count_homozy_ALT,
     ylab = "nombre de sujets",
     freq = TRUE,
     breaks = 100, 
     main = "Distribution des comptes de SNVs \n hétérozygotes",
     cex.lab = 1.5,
     cex.main = 2)
hist(SNV_count_heterozy,
     ylab = "nombre de sujets",
     freq = TRUE,
     breaks = 100, 
     main = "Distribution des comptes de SNVs \n homoyzgotes pour l'allèle alternatif",
     cex.lab = 1.5,
     cex.main = 2)
suppressWarnings(par(opar))

De même, on peut représenter les boxplots:

In [ ]:
##### cell 26

options(repr.plot.width = 20, repr.plot.height = 10)
# On the same scale
boxplot(SNV_count_zygo_df)

# to each it's own
opar <- par()
par(mfrow = c(1, 3))
boxplot(SNV_count_zygo_df$SNV_count_homozy_REF,
        main = 'Homozygote REF SNV counts')

boxplot(SNV_count_zygo_df$SNV_count_heterozy,
        main = 'Heterozygote SNV counts')

boxplot(SNV_count_zygo_df$SNV_count_homozy_ALT,
        main = 'Homozygote ALT SNV counts')

suppressWarnings(par(opar))

Comme les 3 variables  ne sont pas indépendantes, on peut juste regarder la distribution des 2 dernières, donnant une indication plus claire du nombre moyen d'allèles alternatifs sur l'ensemble des sujets.

<div class="alert alert-warning"> 
    <b>Question 5) </b>:
    <br>
    &emsp;Complétez la commande suivante pour faire apparaître le boxplot <b>sans</b> la catégorie <code>SNV_count_homozy_REF</code>. Sur moodle, indiquez ce qui remplace XXX (pas toute la commande). Plusieurs écritures sont possibles.
</div>

In [ ]:
##### cell 27

# Without the homozy_ref, just heterozy and homozy_ALT
boxplot(SNV_count_zygo_df$SNV_count_heterozy,
        XXX,
        names = c('Heterozygote', 'Homozygote ALT'))

On voit qu'en moyenne, il y a environ 70 variants hétérozygotes et 30 variants homozygotes. En génétique des populations, un pourcentage élevé de variants homozygotes peut indiquer la présence de populations consanguines. La distribution bimodale des comptes de variants homozygotes peut aussi suggérer qu'il y a des différences de populations parmi les sujets.

- Nous allons à présent compter les **effectifs de chaque type de génotype pour l'ensemble des individus par SNV** :

Cette fois-ci on applique la fonction `apply()` avec l'argument `MARGIN = 1` permettant d'appliquer une même fonction sur chaque colonne de l'objet X, ici générant une variable pour chaque génotype, contenant le comptes de sujets par SNV.

In [ ]:
##### cell 28

# A quick peek at genotypes
#Overall
individual_count_homozy_REF <- apply( X = brca1_genotypes,
                    MARGIN = 1,
                    FUN = function(x) sum(x == 0)
                                     )
                                     
individual_count_heterozy <- apply(X = brca1_genotypes,
                  MARGIN = 1,
                  FUN = function(x) sum(x == 1)
                                   )
                  
individual_count_homozy_ALT <- apply(X = brca1_genotypes,
                    MARGIN = 1,
                    FUN = function(x) sum(x == 2)
                                     )
                    

Nous regroupons ces résultats dans un nouveau dataframe **individual_count_zygo_df**, avec chaque type de génotype dans une colonne comme une variable quantitative séparée. Chaque ligne correspond ici à un SNV.

In [ ]:
##### cell 29
individual_count_zygo_df <- data.frame(individual_count_homozy_REF,
                                       individual_count_heterozy,
                                       individual_count_homozy_ALT)
str(individual_count_zygo_df)
head(individual_count_zygo_df)

A noter que pour chaque ligne, la somme des comptes (calculée avec la fonction `rowSums()`) donne le nombre d'individus, et que cette somme est la même pour chaque ligne comme l'indique le résumé ci-dessous.

In [ ]:
##### cell 30
summary(rowSums(individual_count_zygo_df))

Dans le dataframe `individual_count_zygo_df`, chaque colonne est une variable discrète. Elles sont non indépendantes car leur somme correspond au nombre de variants. Compte-tenu du nombre élevé d'observations possibles, on peut assimiler là encore ces variables discrètes à des variables variables quantitatives continues et explorer leur distribution.

In [ ]:
##### cell 31
summary(individual_count_zygo_df)

options(repr.plot.width = 20, repr.plot.height = 10)
opar <- par()
par(mfrow = c(1, 3))
hist(individual_count_zygo_df$individual_count_homozy_REF,
     ylab = "nombre de variants",
     freq = TRUE,
     breaks = 100,
     main = "Distribution des comptes de sujets \n homoyzgotes pour l'allèle de référence",
     cex.lab = 1.5,
     cex.main = 2)
hist(individual_count_zygo_df$individual_count_heterozy,
     ylab = "nombre de variants",
     freq = TRUE,
     breaks = 100, 
     main = "Distribution des comptes de sujets \n hétérozygotes",
     cex.lab = 1.5,
     cex.main = 2)
hist(individual_count_zygo_df$individual_count_homozy_ALT,
     ylab = "nombre de variants",
     freq = TRUE,
     breaks = 100, 
     main = "Distribution des comptes de sujets \n homoyzgotes pour l'allèle alternatif",
     cex.lab = 1.5,
     cex.main = 2)
suppressWarnings(par(opar))

In [ ]:
##### cell 32

# all types of genotypes
boxplot(individual_count_zygo_df,
        names = c('Homozyygote REF', 'Heterozygote', 'Homozygote ALT'),
        outline = TRUE)

# Without the homozy_ref, just heterozy and homozy_ALT, and without outliers
boxplot(individual_count_zygo_df[,-1],
        names = c('Heterozygote', 'Homozygote ALT'),
        outline = FALSE)

=> On observe globalement un faible nombre de variants présentant un allèle alternatif à l'état homozygote ou hétérozygote. Il existe cependant quelques valeurs aberrantes pour quelques variants potentiellement interessants. En génétique des populations, le **taux d'hétérozygotie** est un indicateur de l'informativité ou du degré de polymorphisme des variants. Il serait interessant de comparer ces catégories de variants en fonction de la fréquence de leur allèle mineur.

## IV. Pathogénicité des variants
---

### IV.A. Scores predictifs

Nous allons ici observer et comparer plusieurs scores prédictifs qui se proposent d'évaluer la pathogénicité d'un variant.
<br>
Nous avons choisi les scores suivants: 
- DANN (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4341060/)
- REVEL (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5065685/)
- VEST_4 (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3665549/)
- CADD (https://cadd.gs.washington.edu/) qui est à ce jour la référence

Afin de comparer ces scores, qui sont des variables quantitatives continues, nous les représentons 2 à 2 sous forme d'un nuage de points et testons leur corrélation 2 à 2.

In [ ]:
##### cell 33

options(repr.plot.width = 20, repr.plot.height = 12)
opar <- par()
par(mfrow = c(2, 3))

# Correlation
# between pathogeneicity scores:
plot(x = brca1_snps$DANN_score,
     y = brca1_snps$REVEL_score,
     xlab = "DANN",
     ylab = "REVEL",
     cex.lab = 1.5)
abline(lm(formula = REVEL_score ~ DANN_score, data = brca1_snps),
       col = 'tomato3')

plot(x = brca1_snps$DANN_score,
     y = brca1_snps$CADD_phred,
     xlab = "DANN",
     ylab = "CADD",
     cex.lab = 1.5)
abline(lm(formula = DANN_score ~ CADD_phred, data = brca1_snps), 
       col = 'tomato3')

plot(x = brca1_snps$DANN_score,
     y = brca1_snps$VEST4_score,
     xlab = "DANN",
     ylab = "VEST_4",
     cex.lab = 1.5)
abline(lm(formula = DANN_score ~ VEST4_score, data = brca1_snps),
       col = 'tomato3')

plot(x = brca1_snps$REVEL_score,
     y = brca1_snps$CADD_phred,
     xlab = "REVEL",
     ylab = "CADD",
     cex.lab = 1.5)
abline(lm(formula = CADD_phred ~ REVEL_score, data = brca1_snps),
       col = 'tomato3')

plot(x = brca1_snps$REVEL_score,
     y = brca1_snps$VEST4_score,
     xlab = "REVEL",
     ylab = "VEST_4",
     cex.lab = 1.5)
abline(lm(formula = VEST4_score ~ REVEL_score, data = brca1_snps), 
       col = 'tomato3')

plot(x = brca1_snps$CADD_phred,
     y = brca1_snps$VEST4_score,
     xlab = "REVEL",
     ylab = "VEST_4",
     cex.lab = 1.5)
abline(lm(formula = VEST4_score ~ REVEL_score, data = brca1_snps),
       col = 'tomato3')


suppressWarnings(par(opar))

In [ ]:
##### cell 34

# Statistical testing:

cat("-----REVEL and DANN:\n")
cor.test(brca1_snps$REVEL_score, brca1_snps$DANN_score, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

cat("\n-----CADD and DANN:\n")
cor.test(brca1_snps$CADD_phred, brca1_snps$DANN_score, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

cat("\n-----VEST4 and DANN:\n")
cor.test(brca1_snps$VEST4_score, brca1_snps$DANN, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

cat("-----CADD and REVEL:\n")
cor.test(brca1_snps$CADD_phred, brca1_snps$REVEL_score, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

cat("\n-----VEST4 and REVEL:\n")
cor.test(brca1_snps$VEST4_score, brca1_snps$REVEL_score, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

cat("\n-----VEST4 and CADD:\n")
cor.test(brca1_snps$VEST4_score, brca1_snps$CADD_phred, 
         alternative = 'two.sided', 
         method = 'pearson', 
         use = 'complete.obs') # complete to eliminate the NA's

On pouvait aussi générer directement la matrice des coefficients de corrélation 2 à 2 et les valeurs p, ainsi que des représentations graphiques avec les fonctions suivantes.

In [ ]:
##### cell 35

# generate pairwise correlation coeffifients of a matrix
cor(brca1_snps[,c("REVEL_score", "DANN_score", "CADD_phred", "VEST4_score")],
    method = 'pearson',
   use = 'complete.obs')

# draw a heatmap of the pairwise correlation matrix
heatmap(cor(brca1_snps[,c("REVEL_score", "DANN_score", "CADD_phred", "VEST4_score")],
    method = 'pearson',
   use = 'complete.obs'))

# do all in one: matrix of correlation coefficients, matrix of pvalues, correlograme
source("http://www.sthda.com/upload/rquery_cormat.r")
options(width = 200)
rquery.cormat(brca1_snps[,c("REVEL_score", "DANN_score", "CADD_phred", "VEST4_score")])




<div class="alert alert-warning"> 
    <b>Question 6) </b>:
    <br>
    &emsp;Commentez brièvement (un court paragraphe) les résultats des <i>cells</i> 33 à 35 et expliquez si ils représentent selon vous un problème.
</div>

### IV.B. Introns, exons et pathogénicité

Les analyses suivantes visent à déterminer s'il existe une différence de pathogénicité pour les variants de _BRCA1_ selon leur localisation dans les exons ou dans les introns.

- distribution des SNVs selon leur position dans le gène:

In [ ]:
##### cell 36

# Subsetting and comparing exonic and intronic patho scores
table(brca1_snps$Func.refGene, useNA = "always")
barplot(proportions (as.matrix(table(brca1_snps$Func.refGene)), 2),
    beside = TRUE,
        legend = sort(unique(brca1_snps$Func.refGene)))

- étude de la pathogénicité selon le score DANN selon la position exonique ou intronique:

<div class="alert alert-warning"> 
    <b>Question 7) </b>:
    <br>
    &emsp;Complétez la <i>cell</i> 37 pour connaitre le nombre de données manquantes pour le score CADD  dans la catégorie <code>is_intronic</code>.
</div>

In [ ]:
##### cell 37

# the next two commands return TRUE or FALSE depending on the result of the test
is_exonic <- brca1_snps$Func.refGene == 'exonic'
is_intronic <- brca1_snps$Func.refGene == 'intronic'

# we can use is.na() to count the TRUEs and sum them
sum(is.na(brca1_snps$CADD_phred[which(is_exonic)]))
sum(is.na(XXX))

In [ ]:
##### cell 38

# since there are no NAs in Func.refGene (cf. cell 36), we can remove which() in subsetting for this variable in the next commands

summary(brca1_snps$CADD_phred[is_exonic], na.rm = TRUE)
summary(brca1_snps$CADD_phred[is_intronic], na.rm = TRUE)

options(repr.plot.width = 8, repr.plot.height = 8)
boxplot(brca1_snps$CADD_phred[is_exonic], brca1_snps$CADD_phred[is_intronic], 
        names = c('exonic', 'intronic'), ylab = 'CADD')

<div class="alert alert-warning"> 
    <b>Question 8) </b>:
    <br>
         &emsp;Pouvez-vous effectuer un test statistique afin de savoir si la différence de pathogénicité prédite par le score CADD selon la localistion exon | intron d'un SNP est significative ?
</div>

## V. Etude des génotypes selon les metadata (géographie et sexe)
---

### V.A. Préparation des données

- Nous nous focalisons sur les **sujets présents à la fois dans `metadata` et dans `brca1_genotypes`**.

Le nombre de sujets en commun est donné par la commande suivante:

In [ ]:
##### cell 39
length(intersect(metadata$SampleID, colnames(brca1_genotypes)))

Nous créons à présent 2 sous-dataframes avec ces seuls sujets en commun:

In [ ]:
##### cell 40

#  metadata and SNPS
brca1_genotypes_filt <- brca1_genotypes[ , colnames(brca1_genotypes) %in% metadata$SampleID]
str(brca1_genotypes_filt)

metadata_filt <- metadata[metadata$SampleID %in% colnames(brca1_genotypes_filt), ]
str(metadata_filt)

- Nous créons ci-dessous des **sous-jeux de données pour les différentes catégories de populations**:
    - Hommes
    - Femmes
    - Afrique (AFR)
    - Asie de l'est (EAS)
    - Asie du sud (SAS)

<div class="alert alert-warning"> 
    <b>Question 9) </b> :
    <br>
         &emsp; Complétez la commande suivante pour créer le vecteur <code>sample_AFR</code>.
</div>

Pour ce faire nous identifions d'abord les sujets appartenant à chaque catégorie de population.

In [ ]:
##### cell 41

# generating a vector with subjects per category

sample_man <- metadata_filt$SampleID[metadata_filt$Sex == 1]

sample_wom <- metadata_filt$SampleID[metadata_filt$Sex == 2]

sample_AFR <- metadata_filt$SampleID[XXX]

sample_EAS <- metadata_filt$SampleID[metadata_filt$Superpopulation == 'EAS']

sample_SAS <- metadata_filt$SampleID[metadata_filt$Superpopulation == 'SAS']

sample_three_pop <- c(sample_AFR, sample_EAS, sample_SAS)

Puis nous extrayons les colonnes correspondantes dans la matrice brca1_genotypes.

In [ ]:
##### cell 42

## Filtering
brca1_man <- brca1_genotypes_filt[ ,sample_man]
brca1_wom <- brca1_genotypes_filt[ ,sample_wom]
brca1_AFR <- brca1_genotypes_filt[ ,sample_AFR]
brca1_EAS <- brca1_genotypes_filt[ ,sample_EAS]
brca1_SAS <- brca1_genotypes_filt[ ,sample_SAS]

### V.B. Comparaison des distributions des fréquences des allèles ALT sur l'ensemble des variants

- On calcule ensuite le **nombre total et la proportion d'allèles alternatifs présents dans les différentes catégories de population** sur l'ensemble des variants.

Nous pouvons déjà compter le nombre de génotypes dans une sous-population, par exemple dans le sous-groupe de femmes.

In [ ]:
##### cell 43

table(brca1_wom)

Et en déduire le nombre d'observations de l'allèle ALT. Attention, les homozygotes en ont 2 copies!

In [ ]:
##### cell 44

86498 +  2 * 38356


On peut utiliser la fonction `sum()` pour directement compter le nombre  d'observations de l'allèle ALT  et en déduire sa fréquence) pour chaque catégorie de la population:

In [ ]:
##### cell 45

# Comparing the number of ALT between population

## Counting
cat("====Counts:")
cat("\n------Women and Men:\n")
sum(brca1_wom)
sum(brca1_man)
cat("------Geographic:\n")
sum(brca1_AFR)
sum(brca1_EAS)
sum(brca1_SAS)

cat("\n====Proportions:")
cat("\n------Women and Men:\n")
round(sum(brca1_wom) / (dim(brca1_wom)[1]*dim(brca1_wom)[2]*2), 4)
round(sum(brca1_man) / (dim(brca1_man)[1]*dim(brca1_man)[2]*2), 4)
cat("------Geographic:\n")
round(sum(brca1_AFR) / (dim(brca1_AFR)[1]*dim(brca1_AFR)[2]*2), 4)
round(sum(brca1_EAS) / (dim(brca1_EAS)[1]*dim(brca1_EAS)[2]*2), 4)
round(sum(brca1_SAS) / (dim(brca1_SAS)[1]*dim(brca1_SAS)[2]*2), 4)

<div class="alert alert-warning"> 
    <b>Question 10) </b>:
    <br>
         &emsp; Commentez brièvement (un ou deux court·s paragraphe·s) les fréquences en fonction des sous-populations.
</div>

### V.C. Comparaison des fréquences de l'allèle ALT pour un SNV d'intérêt

- A titre d'exemple, nous nous focalisons sur le variant [rs8176166](https://www.ncbi.nlm.nih.gov/snp/rs8176166)

In [ ]:
##### cell 46
brca1_snps[which(brca1_snps$avsnp150 == "rs8176166"), 1:20]

- La fonction suivante vous est donnée afin de pouvoir effectuer des tests de ${\chi}^2$ ou des test exact de Fisher sur la présence de SNP d'intérêt entre deux populations.
Ici, le test exact de Fisher nous donnera la même information qu'un test de ${\chi}^2$ mais il est dit _exact_ car le calcul de la valeur _p_ ne dépend pas d'un calcul basé sur un approximation...C'est un test non paramétrique toujours utilisable.

In [ ]:
##### cell 47

# Function for statistical test between two populations
stat_test <- function(matrix_1, matrix_2, wanted_snp, fisher = FALSE) {
  tab_1 <- table(matrix_1[wanted_snp, ])
  tab_2 <- table(matrix_2[wanted_snp, ])
  name_1 <- deparse(substitute(matrix_1))
  name_2 <- deparse(substitute(matrix_2))
  # Replace empty values by zeroes if needed
  if(is.na(tab_1[2])) tab_1[2] <- 0
  if(is.na(tab_2[2])) tab_2[2] <- 0
  if(is.na(tab_1[3])) tab_1[3] <- 0
  if(is.na(tab_2[3])) tab_2[3] <- 0
  
  cont_tab <- rbind(tab_1, tab_2)
  rownames(cont_tab) <- c(name_1, name_2)
  colnames(cont_tab) <- 0:2  
  print(cont_tab)
  
  if(fisher == TRUE) {
    out_fish <- fisher.test(cont_tab)
    return(out_fish)
  }
  
  out_chi <- chisq.test(cont_tab)
  return(out_chi)
}

In [ ]:
##### cell 48
wanted_snp_1 <- brca1_snps$avsnp150 == 'rs8176166'

In [ ]:
##### cell 49
cat("=============== Chisquare tests: ===============\n")
cat("\n----- AFR versus SAS\n")
stat_test(matrix_1 = brca1_AFR, matrix_2 = brca1_SAS, wanted_snp = wanted_snp_1)
cat("\n----- AFR versus EAS\n")
stat_test(matrix_1 = brca1_AFR, matrix_2 = brca1_EAS, wanted_snp = wanted_snp_1)
cat("\n----- SAS versus EAS\n")
stat_test(matrix_1 = brca1_SAS, matrix_2 = brca1_EAS, wanted_snp = wanted_snp_1)
cat("\n=============== Fisher exact tests: ===============\n")
cat("\n----- AFR versus SAS\n")
stat_test(matrix_1 = brca1_AFR, matrix_2 = brca1_SAS, wanted_snp = wanted_snp_1, fisher = TRUE)
cat("\n----- AFR versus EAS\n")
stat_test(matrix_1 = brca1_AFR, matrix_2 = brca1_EAS, wanted_snp = wanted_snp_1, fisher = TRUE)
cat("\n----- SAS versus EAS\n")
stat_test(matrix_1 = brca1_SAS, matrix_2 = brca1_EAS, wanted_snp = wanted_snp_1, fisher = TRUE)

<div class="alert alert-warning"> 
    <b>Question 10) </b>:
    <br>
         &emsp;En observant les différentes table de contingence et les résultats des tests statistiques, diriez-vous pour le SNP considéré que les affirmations suivantes sont vraies ?:
    <br>
    &emsp; - Les populations <code>AFR</code> et <code>EAS</code> sont plus proches entre elles qu'elles ne le sont de <code>SAS</code>
    <br>
    &emsp; - Les populations <code>EAS</code> et <code>SAS</code> sont plus proches entre elles qu'elles ne le sont de <code>AFR</code>
    <br>
    &emsp; - Ces résultats sont généralisables à l'ensemble des SNP du génome.
    <br>
    &emsp; - le test de Chi^2 entre <code>AFR</code> et <code>EAS</code> envoie un <i>warning message</i> car l'une des catégories présente un faible effectif.
</div>

## VI. Analyse multidimensionnelle
---

La section suivante est un bonus et ne comptera que positiviement dans la note finale.

Regardez la vidéo suivante afin de mieux comprendre la PCA en 5 minutes https://youtu.be/HMOI_lkzW08 !

<div class="alert alert-block alert-danger">
    <b>Attention:</b> 
    La commande suivante peut prendre un peu de temps (environ 5 à 10 minutes) de calcul. Lancez-là pendant que vous regardez la vidéo.
    <br>
    Patience patience ...
</div>

In [ ]:
##### cell 50
# PCA
system.time({
brca1_pca <- 
  FactoMineR::PCA(X = t(brca1_genotypes_filt), ncp = 2, graph = FALSE)
 })

In [ ]:
##### cell 51
brca1_pca_forplot <-
  cbind(brca1_pca$ind$coord[,1:2], metadata_filt)

Représentation des deux premières dimensions (principal components) avec les points (qui représentent donc les individus) colorés en fonction du sexe.

In [ ]:
##### cell 52
ggplot(as.data.frame(brca1_pca_forplot)) +
  geom_point(aes(x = Dim.1, y = Dim.2, colour = as.factor(metadata_filt$Sex))) +
  scale_colour_viridis_d() +
  theme_bw()

Représentation des deux premières dimensions (principal components) avec les points (qui représentent donc les individus) colorés en fonction de la population.

In [ ]:
##### cell 53
ggplot(as.data.frame(brca1_pca_forplot)) +
  geom_point(aes(x = Dim.1, y = Dim.2, colour = Superpopulation), alpha = 0.5) +
  scale_colour_viridis_d() +
  theme_bw()

On approfondit avec le paquet factoextra:

In [ ]:
##### cell 54
factoextra::fviz_eig(brca1_pca, addlabels = TRUE)

In [ ]:
##### cell 55
factoextra::fviz_pca_ind(brca1_pca, label = "none", habillage = as.factor(metadata_filt$Sex),
             addEllipses = TRUE, ellipse.level = 0.95)

In [ ]:
##### cell 56
factoextra::fviz_pca_ind(brca1_pca, label = "none", habillage = as.factor(metadata_filt$Superpopulation),
             addEllipses = TRUE, ellipse.level = 0.95)

<div class="alert alert-warning"> 
    <b>Question bonus) </b>:
    <br>
         &emsp; Décrivez et essayez d'interpreter les graphiques issus de cette PCA.
</div>

*[Last edition: 19/04/2023 by CVandiedonck]*